#GPU_GLCM_Calc:-

In [ ]:
# Install CuPy for GPU accelerated operations (adjust the version if needed).
# !pip install cupy-cuda11x

# Import required libraries
import cv2
import glob
import numpy as np
import cupy as cp
from google.colab import drive
from tqdm.notebook import tqdm
import pandas as pd
import os

# # Mount Google Drive to access your dataset
# drive.mount('/content/drive')

# # Define the base path where the labeled folders are stored on Google Drive.
# # Adjust the base path accordingly.
# base_path = '/content/drive/MyDrive/path_to_images'
base_path = final_path

# List of folder names (labels)
labels = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes',
          'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']

# Collect all image file paths with their associated labels.
image_files = []
for label in labels:
    folder_path = os.path.join(base_path, label, '*.jpg')
    files = glob.glob(folder_path)
    # Save a tuple of (file_path, label) for each image.
    for f in files:
        image_files.append((f, label))

print(f"Found {len(image_files)} images across {len(labels)} labels.")

def quantize_image(image, levels=8):
    """
    Quantize a grayscale image to a fixed number of intensity levels.
    """
    factor = 256 // levels
    return (image // factor).astype(np.uint8)

def compute_glcm_features_for_offset(gray, levels=8, offset=(0,1)):
    """
    Compute the GLCM (for a given offset) and extract features using GPU.

    Parameters:
      gray   : 2D numpy array (quantized grayscale image)
      levels : Number of gray levels (default is 8)
      offset : Tuple (dy, dx) defining the spatial relationship.
               For example, (0,1) corresponds to a horizontal neighbor.

    Returns:
      Dictionary with computed GLCM features.
    """
    # Convert image to a CuPy array
    gpu_img = cp.asarray(gray)
    rows, cols = gpu_img.shape
    dy, dx = offset

    # Determine valid slice indices for the image and its neighbor based on the offset.
    if dy >= 0:
        row_range = slice(0, rows - dy)
        row_range_offset = slice(dy, rows)
    else:
        row_range = slice(-dy, rows)
        row_range_offset = slice(0, rows + dy)

    if dx >= 0:
        col_range = slice(0, cols - dx)
        col_range_offset = slice(dx, cols)
    else:
        col_range = slice(-dx, cols)
        col_range_offset = slice(0, cols + dx)

    # Extract corresponding pixel pairs for the given offset.
    a = gpu_img[row_range, col_range]
    b = gpu_img[row_range_offset, col_range_offset]

    # Flatten the arrays and compute a combined index for the co-occurrence.
    a_flat = a.ravel()
    b_flat = b.ravel()
    indices = a_flat * levels + b_flat

    # Count occurrences using GPU-accelerated bincount.
    glcm_counts = cp.bincount(indices, minlength=levels*levels).reshape(levels, levels)

    # Normalize the GLCM to create a probability matrix.
    glcm_prob = glcm_counts.astype(cp.float32)
    total = cp.sum(glcm_prob)
    if total > 0:
        glcm_prob /= total

    # Create index grids for feature computations.
    i = cp.arange(levels).reshape(-1, 1)
    j = cp.arange(levels).reshape(1, -1)

    # Calculate features:
    contrast = cp.sum((i - j)**2 * glcm_prob)
    energy = cp.sum(glcm_prob**2)
    homogeneity = cp.sum(glcm_prob / (1 + cp.abs(i - j)))

    # For correlation, compute marginal means and standard deviations.
    pi = cp.sum(glcm_prob, axis=1, keepdims=True)
    pj = cp.sum(glcm_prob, axis=0, keepdims=True)
    mu_i = cp.sum(i * pi)
    mu_j = cp.sum(j * pj)
    sigma_i = cp.sqrt(cp.sum(((i - mu_i)**2) * pi))
    sigma_j = cp.sqrt(cp.sum(((j - mu_j)**2) * pj))

    # Avoid division by zero.
    if sigma_i == 0 or sigma_j == 0:
        correlation = cp.array(0.0)
    else:
        correlation = cp.sum(((i - mu_i) * (j - mu_j) * glcm_prob) / (sigma_i * sigma_j))

    return {
        'contrast': float(contrast.get()),
        'energy': float(energy.get()),
        'homogeneity': float(homogeneity.get()),
        'correlation': float(correlation.get())
    }

def compute_multidirectional_glcm_features(image, levels=8, offsets=None):
    """
    Compute and concatenate GLCM features for multiple directions.

    Parameters:
      image   : 2D numpy array (quantized grayscale image)
      levels  : Number of gray levels (default is 8)
      offsets : Dictionary with direction names as keys and offset tuples as values.
                Defaults to four directions: 0°, 45°, 90°, and 135°.

    Returns:
      Dictionary of concatenated features with keys indicating the direction.
    """
    if offsets is None:
        offsets = {
            '0':   (0, 1),   # 0°
            '45':  (-1, 1),  # 45°
            '90':  (-1, 0),  # 90°
            '135': (-1, -1)  # 135°
        }

    features = {}
    # Loop over each direction, compute features, and store with directional keys.
    for direction, off in offsets.items():
        feat = compute_glcm_features_for_offset(image, levels=levels, offset=off)
        for key, value in feat.items():
            features[f"{key}_{direction}"] = value
    return features

# Process each image, compute multi-directional GLCM features, and collect features with label.
features_list = []
for file, label in tqdm(image_files, desc="Processing images"):
    # Read the image in grayscale
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue  # Skip images that fail to load

    # Quantize the image to reduce intensity levels (default is 8 levels)
    img_q = quantize_image(img, levels=8)

    # Compute concatenated multi-directional GLCM features.
    feats = compute_multidirectional_glcm_features(img_q, levels=8)
    feats['filename'] = file
    feats['label'] = label  # Add the folder/label information
    features_list.append(feats)

# Convert the list of feature dictionaries into a Pandas DataFrame.
df_features = pd.DataFrame(features_list)
print(df_features.head())


#GPU_GLRM_Calc:-

In [ ]:
import numpy as np
import cv2
import os
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# ---------------------------
# GPU-accelerated helper functions
# ---------------------------

def quantize_image_gpu(d_img, levels=64):
    """Quantize image (a CuPy array) to specified gray levels on the GPU."""
    max_val = cp.max(d_img)
    quantized = cp.floor_divide(d_img, (max_val + 1) // levels)
    return quantized.astype(cp.uint8)

# CUDA kernel for horizontal (0°) run-length computation
horizontal_kernel_code = r'''
extern "C" __global__
void compute_glrlm_horizontal(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < height) {
        int offset = row * width;
        unsigned char current = img[offset];
        int run_length = 1;
        for (int j = 1; j < width; j++) {
            unsigned char pixel = img[offset + j];
            if (pixel == current) {
                run_length++;
            } else {
                int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
                atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
                current = pixel;
                run_length = 1;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for vertical (90°) run-length computation
vertical_kernel_code = r'''
extern "C" __global__
void compute_glrlm_vertical(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (col < width) {
        unsigned char current = img[col]; // first row at this column
        int run_length = 1;
        for (int i = 1; i < height; i++) {
            unsigned char pixel = img[i * width + col];
            if (pixel == current) {
                run_length++;
            } else {
                int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
                atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
                current = pixel;
                run_length = 1;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for 45° (diagonal upward-right) computation
diagonal45_kernel_code = r'''
extern "C" __global__
void compute_glrlm_45(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = height * width;
    if (idx < total) {
        int i = idx / width;
        int j = idx % width;
        unsigned char current = img[i * width + j];
        int run_length = 1;
        int x = i - 1;
        int y = j + 1;
        while (x >= 0 && y < width) {
            if (img[x * width + y] == current) {
                run_length++;
                x--;
                y++;
            } else {
                break;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for 135° (diagonal downward-right) computation
diagonal135_kernel_code = r'''
extern "C" __global__
void compute_glrlm_135(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = height * width;
    if (idx < total) {
        int i = idx / width;
        int j = idx % width;
        unsigned char current = img[i * width + j];
        int run_length = 1;
        int x = i + 1;
        int y = j + 1;
        while (x < height && y < width) {
            if (img[x * width + y] == current) {
                run_length++;
                x++;
                y++;
            } else {
                break;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# Compile the kernels
horizontal_kernel = cp.RawKernel(horizontal_kernel_code, 'compute_glrlm_horizontal')
vertical_kernel = cp.RawKernel(vertical_kernel_code, 'compute_glrlm_vertical')
diagonal45_kernel = cp.RawKernel(diagonal45_kernel_code, 'compute_glrlm_45')
diagonal135_kernel = cp.RawKernel(diagonal135_kernel_code, 'compute_glrlm_135')

def compute_glrlm_gpu(d_img, direction):
    """
    Compute the GLRLM on the GPU for the given direction.
    d_img: quantized image (CuPy array, uint8) with shape (height, width)
    direction: one of {0, 45, 90, 135}
    Returns a CuPy array representing the GLRLM.
    """
    height, width = d_img.shape
    levels = int(cp.max(d_img).get()) + 1
    max_run_length = max(height, width)  # maximum possible run length
    d_glrlm = cp.zeros((levels, max_run_length), dtype=cp.uint32)

    if direction == 0:  # Horizontal
        threads = 32
        blocks = (height + threads - 1) // threads
        horizontal_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 90:  # Vertical
        threads = 32
        blocks = (width + threads - 1) // threads
        vertical_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 45:  # Diagonal (45°)
        total = height * width
        threads = 256
        blocks = (total + threads - 1) // threads
        diagonal45_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 135:  # Diagonal (135°)
        total = height * width
        threads = 256
        blocks = (total + threads - 1) // threads
        diagonal135_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    else:
        raise ValueError("Direction must be one of {0, 45, 90, 135}")

    cp.cuda.Stream.null.synchronize()
    return d_glrlm

def extract_glrlm_features_gpu(d_glrlm):
    """Extract GLRLM features on the GPU and return them as a list of floats."""
    total_runs = cp.sum(d_glrlm)
    run_indices = cp.arange(1, d_glrlm.shape[1] + 1, dtype=cp.float32)
    sre = cp.sum(d_glrlm / (run_indices**2)[None, :]) / total_runs
    lre = cp.sum(d_glrlm * (run_indices**2)[None, :]) / total_runs
    gln = cp.sum(cp.sum(d_glrlm, axis=1)**2) / (total_runs**2)
    rln = cp.sum(cp.sum(d_glrlm, axis=0)**2) / (total_runs**2)
    return [float(sre.get()), float(lre.get()), float(gln.get()), float(rln.get())]

def extract_features_from_image_gpu(img_path):
    """
    Read an image from disk, transfer it to the GPU, quantize it, compute the GLRLM
    for each direction, extract features from each, and concatenate all features.
    """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    d_img = cp.asarray(img)
    # Quantize image on GPU (using 16 levels for feature extraction)
    d_img = quantize_image_gpu(d_img, levels=16)

    all_features = []
    for direction in [0, 45, 90, 135]:
        d_glrlm = compute_glrlm_gpu(d_img, direction)
        features = extract_glrlm_features_gpu(d_glrlm)
        all_features += features  # concatenate features from each direction
    return all_features

# ---------------------
# Dataset Preparation
# ---------------------

# Set your dataset path and categories accordingly
dataset_path = "garbage_classification/garbage_classification"  # Change to your dataset location
# Make sure to define CATEGORIES as a list of class names
CATEGORIES = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']

features_list = []
labels_list = []

print(f"Dataset path: {dataset_path}")
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}. Please check the path.")

print("Subdirectories:", os.listdir(dataset_path))

# Process each class
for class_idx, class_name in enumerate(CATEGORIES):
    class_dir = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_dir):
        print(f"Warning: Class directory '{class_dir}' not found. Skipping this class.")
        continue
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        if not os.path.isfile(img_path):
            print(f"Warning: File '{img_path}' not found. Skipping this file.")
            continue
        features_list.append(extract_features_from_image_gpu(img_path))
        labels_list.append(class_idx)

# Convert features and labels to numpy arrays (on CPU)
X = np.array(features_list)
y = np.array(labels_list)

# ---------------------
# Prepare data for Neural Network Training
# ---------------------

# One-hot encode the labels
y_cat = to_categorical(y, num_classes=len(CATEGORIES))
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# ---------------------
# Build a Multilayer Neural Network (MLP)
# ---------------------

input_dim = X_train.shape[1]  # should be 16 features (4 features x 4 directions)

model = Sequential([
    Dense(64, activation='relu', input_dim=input_dim),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ---------------------
# Train the Neural Network
# ---------------------

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=16,
                    validation_data=(X_test, y_test),
                    verbose=1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

# Display detailed classification report
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CATEGORIES))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# ---------------------
# Example Prediction Function
# ---------------------

def predict_texture(img_path):
    feat = extract_features_from_image_gpu(img_path)
    feat = np.array(feat).reshape(1, -1)
    pred_probs = model.predict(feat)
    return CATEGORIES[np.argmax(pred_probs)]

# Example usage:
# print(predict_texture("path/to/your/image.jpg"))


#Singleimg_GPU_GLRM_Calc:-

In [ ]:
import numpy as np
import cv2
import os
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ---------------------------
# GPU-accelerated helper functions
# ---------------------------

def quantize_image_gpu(d_img, levels=64):
    """Quantize image (a CuPy array) to specified gray levels on the GPU."""
    max_val = cp.max(d_img)
    quantized = cp.floor_divide(d_img, (max_val + 1) // levels)
    return quantized.astype(cp.uint8)

# CUDA kernel for horizontal (0°) run-length computation
horizontal_kernel_code = r'''
extern "C" __global__
void compute_glrlm_horizontal(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < height) {
        int offset = row * width;
        unsigned char current = img[offset];
        int run_length = 1;
        for (int j = 1; j < width; j++) {
            unsigned char pixel = img[offset + j];
            if (pixel == current) {
                run_length++;
            } else {
                int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
                atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
                current = pixel;
                run_length = 1;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for vertical (90°) run-length computation
vertical_kernel_code = r'''
extern "C" __global__
void compute_glrlm_vertical(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (col < width) {
        unsigned char current = img[col]; // first row at this column
        int run_length = 1;
        for (int i = 1; i < height; i++) {
            unsigned char pixel = img[i * width + col];
            if (pixel == current) {
                run_length++;
            } else {
                int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
                atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
                current = pixel;
                run_length = 1;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for 45° (diagonal upward-right) computation
diagonal45_kernel_code = r'''
extern "C" __global__
void compute_glrlm_45(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = height * width;
    if (idx < total) {
        int i = idx / width;
        int j = idx % width;
        unsigned char current = img[i * width + j];
        int run_length = 1;
        int x = i - 1;
        int y = j + 1;
        while (x >= 0 && y < width) {
            if (img[x * width + y] == current) {
                run_length++;
                x--;
                y++;
            } else {
                break;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# CUDA kernel for 135° (diagonal downward-right) computation
diagonal135_kernel_code = r'''
extern "C" __global__
void compute_glrlm_135(const unsigned char* img, int height, int width, int glrlm_width, unsigned int* glrlm) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = height * width;
    if (idx < total) {
        int i = idx / width;
        int j = idx % width;
        unsigned char current = img[i * width + j];
        int run_length = 1;
        int x = i + 1;
        int y = j + 1;
        while (x < height && y < width) {
            if (img[x * width + y] == current) {
                run_length++;
                x++;
                y++;
            } else {
                break;
            }
        }
        int capped_run = run_length < glrlm_width ? run_length : glrlm_width - 1;
        atomicAdd(&glrlm[current * glrlm_width + capped_run], 1);
    }
}
'''

# Compile the kernels
horizontal_kernel = cp.RawKernel(horizontal_kernel_code, 'compute_glrlm_horizontal')
vertical_kernel = cp.RawKernel(vertical_kernel_code, 'compute_glrlm_vertical')
diagonal45_kernel = cp.RawKernel(diagonal45_kernel_code, 'compute_glrlm_45')
diagonal135_kernel = cp.RawKernel(diagonal135_kernel_code, 'compute_glrlm_135')

def compute_glrlm_gpu(d_img, direction):
    """
    Compute the GLRLM on the GPU for the given direction.
    d_img: quantized image (CuPy array, uint8) with shape (height, width)
    direction: one of {0, 45, 90, 135}
    Returns a CuPy array representing the GLRLM.
    """
    height, width = d_img.shape
    levels = int(cp.max(d_img).get()) + 1
    max_run_length = max(height, width)
    d_glrlm = cp.zeros((levels, max_run_length), dtype=cp.uint32)

    if direction == 0:
        threads = 32
        blocks = (height + threads - 1) // threads
        horizontal_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 90:
        threads = 32
        blocks = (width + threads - 1) // threads
        vertical_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 45:
        total = height * width
        threads = 256
        blocks = (total + threads - 1) // threads
        diagonal45_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    elif direction == 135:
        total = height * width
        threads = 256
        blocks = (total + threads - 1) // threads
        diagonal135_kernel((blocks,), (threads,), (d_img, height, width, max_run_length, d_glrlm))
    else:
        raise ValueError("Direction must be one of {0, 45, 90, 135}")

    cp.cuda.Stream.null.synchronize()
    return d_glrlm

def extract_glrlm_features_gpu(d_glrlm):
    """Extract GLRLM features on the GPU and return them as a list of floats."""
    total_runs = cp.sum(d_glrlm)
    run_indices = cp.arange(1, d_glrlm.shape[1] + 1, dtype=cp.float32)
    sre = cp.sum(d_glrlm / (run_indices**2)[None, :]) / total_runs
    lre = cp.sum(d_glrlm * (run_indices**2)[None, :]) / total_runs
    gln = cp.sum(cp.sum(d_glrlm, axis=1)**2) / (total_runs**2)
    rln = cp.sum(cp.sum(d_glrlm, axis=0)**2) / (total_runs**2)
    return [float(sre.get()), float(lre.get()), float(gln.get()), float(rln.get())]

def extract_features_from_image_gpu(img_path):
    """
    Read a single image from disk, transfer it to the GPU, quantize it, compute the GLRLM
    for each direction, extract features from each, and concatenate all features.
    """
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    d_img = cp.asarray(img)
    d_img = quantize_image_gpu(d_img, levels=16)

    all_features = []
    for direction in [0, 45, 90, 135]:
        d_glrlm = compute_glrlm_gpu(d_img, direction)
        features = extract_glrlm_features_gpu(d_glrlm)
        all_features += features
    return all_features


#Singleimg_GPU_GLCM_Calc:-

#Singleimg_non_GPU_CM

In [ ]:
import cv2
import numpy as np
import argparse


def quantize_image(image: np.ndarray, levels: int = 8) -> np.ndarray:
    """
    Quantize a grayscale image into a fixed number of gray levels.

    Parameters:
      image  : 2D numpy array (uint8 grayscale image)
      levels : Number of gray levels to reduce to (default 8)

    Returns:
      Quantized image as uint8 array with values in [0, levels-1].
    """
    factor = 256 // levels
    return (image // factor).astype(np.uint8)


def compute_glcm_matrix(
    gray: np.ndarray,
    levels: int = 8,
    dx: int = 1,
    dy: int = 0
) -> np.ndarray:
    """
    Compute the Gray-Level Co-occurrence Matrix (GLCM) for a single offset.

    Parameters:
      gray   : 2D numpy array (quantized grayscale image)
      levels : Number of gray levels (default 8)
      dx, dy : Offsets defining spatial relationship

    Returns:
      Normalized GLCM as a 2D numpy array of shape (levels, levels).
    """
    rows, cols = gray.shape
    glcm = np.zeros((levels, levels), dtype=np.uint64)

    # Define valid index ranges based on offset
    i_start, i_end = (-dy, rows) if dy < 0 else (0, rows - dy)
    j_start, j_end = (-dx, cols) if dx < 0 else (0, cols - dx)

    # Accumulate co-occurrences
    for i in range(i_start, i_end):
        for j in range(j_start, j_end):
            r = gray[i, j]
            c = gray[i + dy, j + dx]
            glcm[r, c] += 1

    # Normalize
    total = glcm.sum()
    return glcm.astype(np.float64) / total if total > 0 else glcm.astype(np.float64)


def extract_glcm_features(glcm: np.ndarray) -> dict:
    """
    Compute texture features from a normalized GLCM.

    Features:
      - contrast
      - energy
      - homogeneity
      - correlation

    Returns:
      Dict of feature names to values.
    """
    levels = glcm.shape[0]
    i = np.arange(levels).reshape(-1, 1)
    j = np.arange(levels).reshape(1, -1)

    # Contrast
    contrast = np.sum((i - j)**2 * glcm)
    # Energy (Angular Second Moment)
    energy = np.sum(glcm**2)
    # Homogeneity
    homogeneity = np.sum(glcm / (1 + np.abs(i - j)))
    # Correlation
    pi = glcm.sum(axis=1, keepdims=True)
    pj = glcm.sum(axis=0, keepdims=True)
    mu_i = np.sum(i * pi)
    mu_j = np.sum(j * pj)
    sigma_i = np.sqrt(np.sum(((i - mu_i)**2) * pi))
    sigma_j = np.sqrt(np.sum(((j - mu_j)**2) * pj))

    if sigma_i > 0 and sigma_j > 0:
        correlation = np.sum((i - mu_i) * (j - mu_j) * glcm) / (sigma_i * sigma_j)
    else:
        correlation = 0.0

    return {
        'contrast': float(contrast),
        'energy': float(energy),
        'homogeneity': float(homogeneity),
        'correlation': float(correlation)
    }


def compute_multidirectional_glcm_features(
    image: np.ndarray,
    levels: int = 8,
    offsets: dict = None
) -> dict:
    """
    Compute GLCM features for multiple directions.

    Parameters:
      image   : 2D numpy array (quantized grayscale image)
      levels  : Number of gray levels
      offsets : Dict mapping direction names to (dy, dx) tuples
                Defaults to {'0':(0,1),'45':(-1,1),'90':(-1,0),'135':(-1,-1)}

    Returns:
      Dict of feature_<direction> entries for each metric and direction.
    """
    if offsets is None:
        offsets = {
            '0°':   (0, 1),
            '45°':  (-1, 1),
            '90°':  (-1, 0),
            '135°': (-1, -1)
        }
    features = {}
    for angle, (dy, dx) in offsets.items():
        glcm = compute_glcm_matrix(image, levels, dx=dx, dy=dy)
        feats = extract_glcm_features(glcm)
        for name, val in feats.items():
            features[f"{name}_{angle}"] = val
    return features


def parse_args():
    parser = argparse.ArgumentParser(
        description='Compute GLCM texture features for a single image.'
    )
    parser.add_argument('image_path', help='Path to the grayscale image file')
    parser.add_argument(
        '--levels', type=int, default=8,
        help='Number of gray levels for quantization (default: 8)'
    )
    return parser.parse_args()



In [ ]:


image_path = 'battery124.jpg'
levels = 8
# args = parse_args()

# Load image in grayscale
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not open image at {image_path}")

# Quantize image
img_q = quantize_image(img, levels)

# Compute features
features = compute_multidirectional_glcm_features(img_q, levels)

# Print features
print("GLCM texture features:")
for k, v in features.items():
    print(f"{k}: {v:.6f}")


GLCM texture features:
contrast_0°: 0.522646
energy_0°: 0.238250
homogeneity_0°: 0.933635
correlation_0°: 0.971874
contrast_45°: 1.034147
energy_45°: 0.217632
homogeneity_45°: 0.889660
correlation_45°: 0.944247
contrast_90°: 0.713725
energy_90°: 0.227567
homogeneity_90°: 0.911987
correlation_90°: 0.961561
contrast_135°: 1.041624
energy_135°: 0.217717
homogeneity_135°: 0.891046
correlation_135°: 0.943844


In [ ]:
arr=[i for i in features.values()]
arr = np.array(arr)
arr.shape

(16,)